In [113]:
import numpy as np
import pandas as pd
import matplotlib as plt
import random
from sklearn import model_selection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aprendizaje-automatico-dc-uba-ar/material/blob/main/tp/01_aprendizaje_supervisado/tp01-enunciado.ipynb)

# Trabajo Práctico -  Aprendizaje supervisado
### Clasificación de expresiones genómicas

<span style="color: red;">**Fecha de entrega: 01 de Mayo del 2024 - hasta las 23:55hs.**

<span style="color: red;">**Fecha de entrega intermedia: 25 de Abril del 2024 - hasta las 23:55hs.**
</span>

## Introducción

En el mundo actual, distintas disciplinas científicas empiezan, cada vez más, a interactuar con el fin de potenciar sus descubrimientos. En este caso dos grupos de investigación de [CONICET](https://www.conicet.gov.ar/) se embarcan en la combinación entre biología y informática para abordar la detección temprana y el pronóstico preciso de enfermedades como el cáncer. Este proyecto combina las tecnologías de secuenciación de nueva generación ([_NGS_](https://es.wikipedia.org/wiki/Secuenciaci%C3%B3n_paralela_masiva), por sus siglas en inglés) con la potencia de la inteligencia artificial. El enfoque se centra en un dataset único que abarca mediciones de [_ARN_](https://es.wikipedia.org/wiki/ARN_mensajero) de 200 [_genes_](https://es.wikipedia.org/wiki/Gen), recopiladas de pacientes con lesiones [_pre-tumorales_](https://en.wikipedia.org/wiki/Hyperplasia). Este conjunto de datos se convierte en una valiosa fuente de información para entender cómo las células en estado de hiperplasia pueden evolucionar hacia [_tumores malignos_](https://en.wikipedia.org/wiki/Neoplasm), una transformación que ha desconcertado a la ciencia durante décadas.

La hiperplasia, es un fenómeno en el que las células experimentan un crecimiento anormal y descontrolado, es un punto de partida crucial en nuestro análisis. ¿Cómo y por qué algunas células que experimentan hiperplasia se convierten en células cancerosas, mientras que otras no? Esta pregunta es el corazón de nuestra investigación. Para responderla se realizo un estudio donde se obtuvieron muestras de distintos tipos de hiperplasias de pacientes con antecedentes familiares y lesiones pre tumorales. Este grupo de pacientes, o cohorte, fue monitoreado periodicamente durante los siguientes 5 años buscando indicios de neoplasias o nuevas hiperplasias más agresivas. Con las muestras obtenidas en este estudio se realizo un [_biobanco_](https://en.wikipedia.org/wiki/Biobank) con las mediciones que habitualmente se hacen en la construccion de este tipo de [_plataformas_](https://xena.ucsc.edu/). Cada muestra fue etiquetada como **_buen pronostico_**, si no hubo indicios de nuevas hiperplasias o similares; contrariamente se etiquetaron como de **_mal pronostico_** si hubo una recaida.

Este trabajo se concentra en un panel de genes, especificamente en la expresion de 200 genes que se creen tienen un papel crucial en la transformacion tumoral y su etiqueta correspondiente.

En concreto:

Tendrán un archivo `.csv` en donde se almacenan:
  - una matriz de datos `X` de $500$ filas en donde cada fila $x^{(i)}$ representa un vector de $200$ características de cada instancia. Es decir, $\textbf{x}^{(i)} = x_1^{(i)}, \dots, x_{200}^{(i)}$ con $i$ entre $1$ y $500$.
  - una columna llamada `target` que representa un vector de $500$ posiciones con dos posibles valores: `True` (ó 1, es decir, tiene buen pronostico) y `False` (ó 0, tiene mal pronostico).

Los datos están en esta [carpeta](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

Por otra parte, tendrán disponibles un conjunto de instancias sin etiquetas, que utilizaremos para comprobar la calidad de sus resultados (ver Ejercicio 5). 

**Recomendamos fuertemente leer primero todo el enunciado del trabajo antes de empezar a trabajar sobre el problema propuesto.**

---

### Sobre el informe

Para este trabajo deberán entregar, además del código de las pruebas y experimentos que realicen, un informe en el que deberan seleccionar, para cada apartado, sus resultados acompañado de un texto que explique, reflexione, justifique y conluya dicho contenido. 

Cada ejercicio indica el largo máximo del texto que se puede incluir. Los gráficos no están contados en dicho espacio. 
Cada gráfico incluido debe contar con:
  
  - nombres de los ejes,
  - título,
  - leyenda autocontenida,
  - debe ser referenciado desde el texto, ya que su inclusión se da porque aporta a la discusión del trabajo.

---

## Ejercicio 1 

### Separación de datos (máximo 2 carillas del informe)

Contarán con una cantidad limitada de datos, por lo cual es importante tomar una buena decisión en el momento de empezar a utilizarlos. 

Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. 

**Importante**: en este punto no está permitido dividir la base de datos utilizando la función `train_test_split` de sklearn. Deben decidir e implementar la separación.

Primero cargamos los datos:

In [23]:
df = pd.read_csv("data.csv")

Veamos la proporción de datos que tienen target = 1 y de los que tienen target = 0.

In [24]:
df["target"].value_counts(normalize=True)

target
0    0.686
1    0.314
Name: proportion, dtype: float64

Elegimos separar los datos con la idea de stratified, es decir, separar los datos pero asegurarnos de que las clases queden balanceadas. Elegimos separar el 85% de los datos para desarrollo (425 datos) y el 15% de los datos (75 datos) para evaluación.

In [25]:
type(round(df["target"].value_counts()[0]*0.85))

int

Implementamos una función que nos separe según lo que queremos:

In [26]:
def stratified_split(df,prop=0.85):
    cant_0 = df["target"].value_counts()[0]
    cant_1 = df["target"].value_counts()[1]
    df_0_indices = df[df["target"]==0].index.to_list() #me quedo con los indices de los que tienen target = 0
    df_1_indices = df[df["target"]==1].index.to_list() #me quedo con los indices de los que tienen target = 1

    random.seed(4)
    prop_df_0 = random.sample(df_0_indices,round(cant_0*prop)) #292
    menos_prop_df_0 = [idx for idx in df_0_indices if idx not in prop_df_0] #51
    prop_df_1 = random.sample(df_1_indices,round(cant_1*prop)) #133
    menos_prop_df_1 = [idx for idx in df_1_indices if idx not in prop_df_1] #24

    desarrollo_indices = prop_df_0+prop_df_1
    evaluacion_indices = menos_prop_df_0+menos_prop_df_1

    desarrollo = df.loc[desarrollo_indices]
    evaluacion = df.loc[evaluacion_indices]
    return desarrollo,evaluacion


Corremos la función:

In [30]:
dev, eval = stratified_split(df)

Veamos la cantidad de datos que quedó en cada uno de los datasets, el de desarrollo y el de evaluación:

In [7]:
print(len(dev))
print(len(eval))

425
75


Ahora veamos que se respeta la proporción original en cada uno de los datasets:

In [8]:
print("Proporción de datos del dataset de desarrollo: \n",dev["target"].value_counts(normalize=True))
print("Proporción de datos del dataset de evaluación: \n",eval["target"].value_counts(normalize=True))

Proporción de datos del dataset de desarrollo: 
 target
0    0.687059
1    0.312941
Name: proportion, dtype: float64
Proporción de datos del dataset de evaluación: 
 target
0    0.68
1    0.32
Name: proportion, dtype: float64


Guardamos los datos de desarrollo y evaluación en un nuevo archivo.

In [ ]:
eval.to_csv("evaluacion.csv",index=False)

In [ ]:
dev.to_csv("desarrollo.csv",index=False)

## Ejercicio 2


### Construcción de modelos (máximo 2 carillas del informe)

Para este punto, la tarea consiste en construir y evaluar modelos de tipo **árbol de decisión**. Además, obtener una **estimación realista de la performance** de los mismos.

1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

1. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_., _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

   En esta oportunidad se va a pedir además de calcular las métricas para cada fold por separado y su promedio, que hagan el cálculo del score global (como vimos en clase), sólo para los folds de validación.
   
   Reportar el resultado en una tabla similar a:

      <table>
      <thead>
      <tr>
      <th align="center">Permutación</th>
      <th>Accuracy (training)</th>
      <th>Accuracy (validación)</th>
      <th>AUPRC (training)</th>
      <th>AUPRC (validación)</th>
      <th>AUC ROC (training)</th>
      <th>AUC ROC (validación)</th>
      </tr>
      </thead>
      <tbody>
      <tr>
      <td align="center">1</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">2</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">3</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">4</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">5</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <tr>
      <td align="center">Promedios</td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      <td></td>
      </tr>
      <td align="center">Global</td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      <td>(NO) </td>
      <td></td>
      </tr>
      </tbody>
      </table>    
  
   **Importante**: de acá en más sólamente utilizaremos el score promedio cuando hagamos _K-fold cross-validation_.
 
1. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

   <table>
   <thead>
   <tr>
   <th align="center">Altura máxima</th>
   <th align="center">Criterio de corte</th>
   <th>Accuracy (training)</th>
   <th>Accuracy (validación)</th>
   </tr>
   </thead>
   <tbody><tr>
   <td align="center">3</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Gini</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">3</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">5</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   <tr>
   <td align="center">Infinito</td>
   <td align="center">Entropía</td>
   <td></td>
   <td></td>
   </tr>
   </tbody></table>


1. ¿Qué conclusiones se pueden sacar de estas tablas?  

#### 1. Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default.

Separamos los datos de desarollo en train y test.

In [3]:
data = pd.read_csv("desarrollo.csv")

In [4]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
                    data[data.columns.difference(['target'])],
                    data['target'],
                    random_state=4,
                    test_size=0.1)

In [5]:
from sklearn.tree import DecisionTreeClassifier
arbol = DecisionTreeClassifier(max_depth=3)

In [6]:
arbol.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [7]:
arbol.score(x_train, y_train)
arbol.score(x_test, y_test)

0.5813953488372093

#### 2. Estimar la performance del modelo utilizando _K-fold cross validation_ con `K=5`, con las métricas _Accuracy_, _Area Under the Precision-Recall Curve (AUPRC)_., _Area Under the Receiver Operating Characteristic Curve (AUCROC)_. 

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, auc, precision_recall_curve, roc_auc_score

In [9]:
def train_tree(X_tr: np.ndarray, y_tr: np.ndarray, tree_params={}) -> DecisionTreeClassifier:
    arbol = DecisionTreeClassifier(**tree_params) #crea el arbol con ciertos hiperparametros que le pasas: la altura maxima 
    arbol.fit(X_tr, y_tr)

    return arbol

def tree_predict(ab: DecisionTreeClassifier, X_test: np.ndarray) -> np.ndarray:
    predictions = ab.predict(X_test) #le pasas el arbol ya entrenado y te devuelve las predicciones del test
    return predictions

def metricas_seleccionadas(y_pred, y_real):
    precision, recall, _ = precision_recall_curve(y_real,y_pred)
    return accuracy_score(y_real, y_pred), auc(precision, recall), roc_auc_score(y_real, y_pred)

In [10]:
#juntar x_train y x_test
x = pd.concat([x_train, x_test])
y = pd.concat([y_train, y_test])

In [11]:
y_pred = np.empty(y.shape) #creamos las predicciones globales
y_pred.fill(np.nan)
accuracy_folds_train = np.array([])
aupcr_folds_train = np.array([])
aucroc_folds_train = np.array([])
accuracy_folds_test = np.array([])
aupcr_folds_test = np.array([])
aucroc_folds_test = np.array([])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)

for train_index, test_index in skf.split(x, y):
    #en train_index estan los indices de los datos que estan en los 5 folds que pertenecen a training
    #en test_index estan los indices de los datos que estan en el unico fold que es el test
    
    #saco el fold que no uso para entrenar
    kf_X_train, kf_X_test = x.iloc[train_index], x.iloc[test_index]
    kf_y_train, kf_y_test = y.iloc[train_index], y.iloc[test_index]

    current_tree = train_tree(kf_X_train, kf_y_train,
                                tree_params={"max_depth":3})
    predictions_train = tree_predict(current_tree, kf_X_train)
    predictions_test = tree_predict(current_tree, kf_X_test)
    y_pred[test_index] = predictions_test #quedan algunos vacios (con NAs) en cada iteracion pero finalmente se llena todo
    
    accuracy_fold_train, aupcr_fold_train, aucroc_fold_train = metricas_seleccionadas(predictions_train, kf_y_train)
    accuracy_fold_test, aupcr_fold_test, aucroc_fold_test = metricas_seleccionadas(predictions_test, kf_y_test)
    
    accuracy_folds_train = np.append(accuracy_folds_train, accuracy_fold_train)
    accuracy_folds_test = np.append(accuracy_folds_test,accuracy_fold_test)
    aupcr_folds_train = np.append(aupcr_folds_train, aupcr_fold_train)
    aupcr_folds_test = np.append(aupcr_folds_test,aupcr_fold_test)
    aucroc_folds_train = np.append(aucroc_folds_train, aucroc_fold_train) 
    aucroc_folds_test = np.append(aucroc_folds_test, aucroc_fold_test)

accuracy_global, aupcr_global, aucroc_global = metricas_seleccionadas(y_pred, y) #mido que tan bien me fue con las predicciones

In [12]:
#crear un dataframe con las metricas
df_metrics = pd.DataFrame({
    "fold": np.arange(1,6),
    "accuracy_train": accuracy_folds_train,
    "accuracy_test": accuracy_folds_test,
    "aupcr_train": aupcr_folds_train,
    "aupcr_test": aupcr_folds_test,
    "aucroc_train": aucroc_folds_train,
    "aucroc_test": aucroc_folds_test
})

promedios = np.array(["promedio"])
for column in df_metrics.columns[1:]:
    promedio = df_metrics[column].mean()
    promedios = np.append(promedios, round(promedio,6))


df_metrics.loc[5] = promedios
#agregar las metricas globales
df_metrics.loc[6] = ["global", np.nan, accuracy_global, np.nan, aupcr_global, np.nan, aucroc_global]

In [13]:
df_metrics

,fold,accuracy_train,accuracy_test,aupcr_train,aupcr_test,aucroc_train,aucroc_test
0,1,0.835294,0.682353,0.460067,0.24724,0.78633,0.6206
1,2,0.835294,0.635294,0.460067,0.144322,0.78633,0.543677
2,3,0.826471,0.694118,0.446895,0.242079,0.775883,0.617497
3,4,0.841176,0.705882,0.469634,0.245316,0.786567,0.61622
4,5,0.838235,0.741176,0.464955,0.349095,0.794751,0.711367
5,promedio,0.835294,0.691765,0.460324,0.24561,0.785972,0.621872
6,global,NaN,0.691765,NaN,0.247725,NaN,0.622155


#### 3. Explorar las siguientes combinaciones de parámetros para  árboles de decisión (siguiendo con $k-fold$ con $k=5$) utilizando [ParameterGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterGrid.html) de _scikit learn_. No está permitido utilizar `GridSearchCV` en este ejercicio.

In [14]:
def grid_search(params):
    y_pred = np.empty(y.shape) #creamos las predicciones globales
    y_pred.fill(np.nan)
    accuracy_folds_train = np.array([])
    accuracy_folds_test = np.array([])
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)

    for train_index, test_index in skf.split(x, y):
        #en train_index estan los indices de los datos que estan en los 5 folds que pertenecen a training
        #en test_index estan los indices de los datos que estan en el unico fold que es el test
        
        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = x.iloc[train_index], x.iloc[test_index]
        kf_y_train, kf_y_test = y.iloc[train_index], y.iloc[test_index]

        current_tree = train_tree(kf_X_train, kf_y_train,
                                    tree_params=params)
        predictions_train = tree_predict(current_tree, kf_X_train)
        predictions_test = tree_predict(current_tree, kf_X_test)
        y_pred[test_index] = predictions_test #quedan algunos vacios (con NAs) en cada iteracion pero finalmente se llena todo
        
        accuracy_fold_train = accuracy_score(kf_y_train, predictions_train)
        accuracy_fold_test = accuracy_score(kf_y_test, predictions_test)
        
        accuracy_folds_train = np.append(accuracy_folds_train, accuracy_fold_train)
        accuracy_folds_test = np.append(accuracy_folds_test,accuracy_fold_test)
    return np.mean(accuracy_folds_train), np.mean(accuracy_folds_test)

In [15]:
from sklearn.model_selection import ParameterGrid

profundidades = [3,5,None]
metricas = ["gini", "entropy"]

def crear_grilla(profundidades, metricas):
    parametros = {
        'max_depth': profundidades,
        'criterion': metricas
    }
    grilla = list(ParameterGrid(parametros))
    return grilla

grilla_parametros = crear_grilla(profundidades, metricas)


# Crear un DataFrame para almacenar los resultados
resultados = pd.DataFrame(columns=['Altura máxima', 'Criterio de corte', 'Accuracy (training)', 'Accuracy (validación)'])
i = 1
for params in grilla_parametros:
    max_depth = params['max_depth']
    criterion = params['criterion']
    accuracy_train, accuracy_val = grid_search(params)
    resultados.loc[i, 'Altura máxima'] = str(max_depth) if max_depth is not None else 'Infinito'
    resultados.loc[i, 'Criterio de corte'] = criterion.capitalize()
    resultados.loc[i, 'Accuracy (training)'] = accuracy_train
    resultados.loc[i, 'Accuracy (validación)'] = accuracy_val
    i = i + 1

In [16]:
resultados

,Altura máxima,Criterio de corte,Accuracy (training),Accuracy (validación)
1,3,Gini,0.835294,0.691765
2,5,Gini,0.939412,0.651765
3,Infinito,Gini,1.0,0.651765
4,3,Entropy,0.805882,0.689412
5,5,Entropy,0.903529,0.670588
6,Infinito,Entropy,1.0,0.701176


#### 4. ¿Qué conclusiones se pueden sacar de estas tablas? 

Observaciones de la primera tabla:
Primero, es importante decir que para las tres métricas siempre dió mejor en el set de training que en el de test. Esto se debe a que al modelo le resulta más fácil predecir datos que usó para entrenarse.
* El accuracy global y el promedio dieron iguales.
* El auroc y el aupcr globales y promedios también dieron muy similares, pero con una leve mejora en el global.
* El aupcr para training para todos los folds dió siempre debajo de 0.35, es decir, muy mal. Lo mismo para aupcr test, dieron todos muy cercanos a 0.46. Esto nos da el indicio de que en general no logramos que tanto precision como recall sean altos al mismo tiempo, pero como estamos en el caso particular de clasificar tumores malignos de benignos, nos interesa especialmente minimizar los falsos positivos (clasificar como benigno cuando en realidad era maligno), es decir maximizar precision. Sería conveniente ver los gráficos.
* El auroc para training dió muy parecido para todos los folds, al rededor de 0.78. Sin embargo, para test podemos ver que hay bastante variabilidad, el mayor tiene 0.711367 y el menor 0.543677. Vemos que nos dió mucho mejor que la aupcr. Como tenemos un dataset desbalanceado, el auroc es mejor métrica al momento de elegir un modelo.

Precision: de las instancias predichas como positivas, ¿qué porcentaje eran positivas?: TP/(TP+FP)
Recall: De las instancias positivas, ¿qué porcentaje fueron predichas como tal?: TP/(TP+FN)

Observaciones de la segunda tabla:
* Podemos ver que en la segunda tabla, cuando hacemos altura = 3 y criterio de corte = 'gini' los valores son de accuracy son iguales que accuracy promedio en la primera tabla (tanto para training como para evaluación), lo cual tiene sentido porque usamos la misma semilla y entonces los folds son los mismos. Además, porque en la primera tabla entrenamos un árbol de altura máxima = 3 y el paramétro por default de criterio es 'gini'.
* Vemos que a medida que aumentamos la altura máxima de los árboles, aumenta el accuracy en el set de training para ambos criterios. Esto se debe a que mientras mayor altura, más se ajusta el modelo a los datos training.
* Se puede observar que el accuracy de validación dió muy parecido para los distintos modelos, a diferencia del de training. Sin embargo, vemos que a diferencia del accuracy de training, no aumenta a medida que aumentamos la altura máxima del árbol. Esto se puede deber a que aumentar la altura del árbol no me ayuda a explicar la relación entre las instancias y la etiqueta, sino que lo que está pasando es que se está ajustando a los datos observados.

## Ejercicio 3

### Comparación de algoritmos 

Se pide explorar distintas combinaciones de algoritmos de aprendizaje con diferentes configuraciones con el objetivo de encontrar el mejor modelo de cada familia de buscar la performance óptima. Para este ejercicio realizar una experimentación utilizando [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Como métrica de performance usar AUCROC resultante de 5-fold cross-validation. 

Algoritmos a probar: 
  - Árboles de decisión
  - KNN (k-vecinos más cercanos)
  - SVM (Support vector machine)
  - LDA (Linear discriminant analysis)
  - Naïve Bayes
  
_Hiperparámetros_: Revisar la documentación de cada uno para la búsqueda de combinaciones prometedoras. 

Documentación extra sobre [`Tuning hyper-parameters`](https://scikit-learn.org/stable/modules/grid_search.html), leer hasta 3.2.2.

Para este ejercicio se pide generar un reporte que contenga: 

1. Una descripción de las distintas combinaciones consideradas y su performance asociada (las que consideren relevantes, con al menos la mejor combinación para cada algoritmo). 

1. Una breve explicación de los factores que creen que produjeron dicho resultado. 

Para cada método pueden incluir hasta media carilla de texto y los gráficos que considere relevantes.

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from scipy.stats import uniform, randint

In [139]:
#arbol
arbol = DecisionTreeClassifier()
dic_arbol = {
    'max_depth': randint(1, 10), #equivale a una uniforme pero para enteros
    'criterion': ['gini', 'entropy', 'log_loss']
}

#knn
knn_classifier = KNeighborsClassifier()
dic_knn = {
    'n_neighbors': randint(1, 20), #numero de vecinos
    'weights': ['uniform', 'distance'], #pesos: cuando hacen la votación pueden tener todos el mismo peso = uniforme, 
                                        #o tener pesos distintos en función de la distancia = distance
    'p': [1, 2] #distancia de minkowski (como se mide la distancia) p = 1 = manhattan y p = 2 = ecuclidiana 
}

#svm
svm_classifier = svm.SVC()
dic_svm = {
    'C': uniform(0, 10), #el C restringe las instancias que pueden caer en el lado incorrecto
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], #el kernel con el que cambiamos la dimension
                                                    #lineal
                                                    #poly: abajo aclaramos el grado
                                                    #rbf: radial (el de los circulos)
                                                    #sigmoid 
    'degree': randint(2, 11) #aclara el grado del polinomico, sino lo ignora
}


#Vemos que el kernel RBF es el que mejor se comporta, entonces hacemos una busqueda mas fina, dejandole mas libertad a C
dic_svm2 = {
    'C': uniform(0, 20), #el C restringe las instancias que pueden caer en el lado incorrecto
    'kernel': ['rbf'], #el kernel con el que cambiamos la dimension
}

#lda
lda_classifier = LinearDiscriminantAnalysis() #no tiene parametros
dic_lda = {}

#naive bayes
naive_bayes_classifier = GaussianNB()
dic_naive_bayes = { #no tiene hiperparametros
    'var_smoothing': uniform(0, 0.001)
    }

In [144]:
def randomized_search(modelo, parametros,x,y, iter):
    random_search = RandomizedSearchCV(estimator=modelo, param_distributions=parametros, n_iter=iter,
                                    scoring='roc_auc', cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), 
                                    verbose=0, random_state=41, n_jobs=-1)
    return random_search.fit(x, y)

Llamemos a la función para cada modelo

In [140]:
random = m(naive_bayes_classifier, dic_naive_bayes, x, y, 5)
#ver todos los parametros que probo
print("\nEl mejor AUROC:")
print(random.best_score_)
print("\nMejores Parámetros:")
print(random.best_params_)

resultados = random.cv_results_
    
    # Convertir los resultados a un DataFrame de Pandas para análisis
df_resultados = pd.DataFrame(resultados)
#for i in range(0,len(df_resultados["params"])):
    #df_resultados.at[i, "params"] = {  "kernel": df_resultados.loc[i, "params"]["kernel"] ,
                                    #"C": round(df_resultados.loc[i, "params"]["C"],3) }


El mejor AUROC:
0.8181442018379481

Mejores Parámetros:
{'var_smoothing': 0.0006768162411105584}


In [141]:
display(df_resultados[['params', 'mean_test_score']])

,params,mean_test_score
0,{'var_smoothing': 0.00025092362374494015},0.810785
1,{'var_smoothing': 4.6095820672611755e-05},0.804231
2,{'var_smoothing': 0.0006768162411105584},0.818144
3,{'var_smoothing': 4.346948564475206e-05},0.804231
4,{'var_smoothing': 0.00011642370316653127},0.806416


In [170]:
modelos = [arbol, knn_classifier, svm_classifier, svm_classifier, lda_classifier, naive_bayes_classifier]
dic = [dic_arbol, dic_knn, dic_svm, dic_svm2, dic_lda, dic_naive_bayes]
cantidad_iteraciones = [10, 10, 10, 10, 1, 10]

top5 = pd.DataFrame(columns=["Modelo", "mean_AUROC", "parametros"])

for i in range(0,6):
    resultados = randomized_search(modelos[i], dic[i], x, y, cantidad_iteraciones[i])
    parametros_probados = pd.DataFrame(resultados.cv_results_)
    parametros_probados = parametros_probados[['params', 'mean_test_score']]
    parametros_ordenados = parametros_probados.sort_values(by='mean_test_score', ascending=False)
    parametros_seleccionados = parametros_ordenados.head(5).copy()  # Crear una copia del DataFrame
    
    parametros_seleccionados["Modelo"] = str(modelos[i])
    parametros_seleccionados = parametros_seleccionados.rename(columns={"mean_test_score": "mean_AUROC", "params": "parametros"})

    top5 = pd.concat([top5, parametros_seleccionados], ignore_index=True)  # Usar ignore_index=True para reindexar el DataFrame resultante


C:\Users\gr586\AppData\Local\Temp\ipykernel_20420\794549084.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  top5 = pd.concat([top5, parametros_seleccionados], ignore_index=True)  # Usar ignore_index=True para reindexar el DataFrame resultante


In [171]:
top5

,Modelo,mean_AUROC,parametros
0,DecisionTreeClassifier(),0.636871,"{'criterion': 'gini', 'max_depth': 3}"
1,DecisionTreeClassifier(),0.632716,"{'criterion': 'log_loss', 'max_depth': 4}"
2,DecisionTreeClassifier(),0.624257,"{'criterion': 'gini', 'max_depth': 2}"
3,DecisionTreeClassifier(),0.616719,"{'criterion': 'log_loss', 'max_depth': 7}"
4,DecisionTreeClassifier(),0.607732,"{'criterion': 'entropy', 'max_depth': 9}"
5,KNeighborsClassifier(),0.839378,"{'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}"
6,KNeighborsClassifier(),0.828620,"{'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}"
7,KNeighborsClassifier(),0.826467,"{'n_neighbors': 12, 'p': 1, 'weights': 'uniform'}"
8,KNeighborsClassifier(),0.794974,"{'n_neighbors': 3, 'p': 1, 'weights': 'distance'}"
9,KNeighborsClassifier(),0.793982,"{'n_neighbors': 18, 'p': 2, 'weights': 'uniform'}"


In [173]:
top5.sort_values(by='mean_AUROC', ascending=False)

,Modelo,mean_AUROC,parametros
15,SVC(),0.892637,"{'C': 5.018472474898803, 'kernel': 'rbf'}"
16,SVC(),0.891240,"{'C': 3.8186132555947827, 'kernel': 'rbf'}"
17,SVC(),0.891211,"{'C': 8.375601738986058, 'kernel': 'rbf'}"
19,SVC(),0.891078,"{'C': 12.077313778976261, 'kernel': 'rbf'}"
10,SVC(),0.891078,"{'C': 7.239741430299464, 'degree': 4, 'kernel'..."
18,SVC(),0.891078,"{'C': 13.536324822211167, 'kernel': 'rbf'}"
11,SVC(),0.858611,"{'C': 0.6952046781160548, 'degree': 7, 'kernel..."
5,KNeighborsClassifier(),0.839378,"{'n_neighbors': 7, 'p': 1, 'weights': 'uniform'}"
12,SVC(),0.828771,"{'C': 3.1467693397954024, 'degree': 7, 'kernel..."
6,KNeighborsClassifier(),0.828620,"{'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}"


## Ejercicio 4: 
### Diagnóstico Sesgo-Varianza.

<span style="color: red;">(no realizar hasta la clase _Sesgo y Varianza_)</span>

En este punto, se pide inspeccionar **tres** de sus mejores modelos encontrados hasta ahora de cada familia de modelos: la mejor configuración para el árbol de decisión, la mejor configuración para LDA y la mejor configuración para SVM. Para ello:

1. Graficar curvas de complejidad para cada modelo (excepto para LDA), variando la profundidad en el caso de árboles, y el hiperparámetro C en el caso de SVM. Diagnosticar cómo afectan al sesgo y a la varianza esos dos hiperparámetros.
2. Graficar curvas de aprendizaje para cada modelo. En base a estas curvas, sacar conclusiones sobre si los algoritmos parecen haber alcanzado su límite, o bien si aumentar la cantidad de datos debería ayudar.
3. Construir un modelo **RandomForest** con 200 árboles. Explorar para qué sirve el hiperparámetro max_features y cómo afecta a la performance del algoritmo mediante una curva de complejidad. Explicar por qué creen que se dieron los resultados obtenidos. Por último, graficar una curva de aprendizaje sobre los parámetros elegidos para determinar si sería útil o no conseguir más datos.


**Atención**: Tener en cuenta que debemos seguir utilizando AUC ROC como métrica para estas curvas.

Para cada método pueden incluir hasta una carilla de texto y los gráficos que considere relevantes.


## Ejercicio 5: 
### Evaluación de performance

- La entrega del trabajo estará acompañada de una evaluación en la cual deberán poner a prueba su mejor modelo y sobre todo, su capacidad para estimar sus resultados. 

- Su tarea será estimar la performance (AUCROC) que tendrá su mejor modelo en datos de evaluación (X_held_out). 

- Para ello, deberán predecir las **probabilidades** de las distintas instancias con su modelo, enviarnos dichas probabilidades junto a una estimación con 4 decimales de cuál será el AUCROC resultante y calcularemos el resultado real. Consideraremos que el **mejor modelo será el que se encuentre más cerca del valor real que calcularemos luego de la entrega**.

- Recomendamos no perder de vista esta evaluación/competencia durante el desarrollo del TP, sobretodo en el momento de separar los datos en los primeros puntos. 

- Para que podamos evaluar la performance, junto con la entrega del informe, deberán enviar un archivo con el numero de grupo con dos digitos en formato csv con la columna `output` y el valor obtenido con 4 decimales (se subirá un ejemplo cuando se publiquen los datos de la competencia) y un valor esperado de AUCROC: `GG_y_pred_held_out_AUCROC`. 

    - Ej.: el grupo tres cree que obtuvo un valor de 0.7321 de AUCROC deberá submitear un archivo llamado: `03_y_pred_held_out_7321.csv`.

- Los datos podrán encontrarlos en este [link](https://github.com/aprendizaje-automatico-dc-uba-ar/material/tree/main/tp/01_aprendizaje_supervisado/datos).

- Las decisiones de este punto pueden desarrollarse hasta en una carilla, aunque con media debería alcanzar.


## Ejercicio 6: 
### Conclusiones

Dar en a lo sumo 2 carillas una conclusión del trabajo realizado, incluyendo problemas encontrados y 
dimensiones no incluidas en el enunciado que hayan sido abordadas durante el desarrollo.

---
## Entregables
- Contarán con un esqueleto en formato Jupyter Notebook en donde podrán intercalar celdas para reportar y responder a los ítems de cada ejercicio. 
- Los entregrables serán
    - Un informe en formato .pdf (**digital**) que responda a los ítems de este enunciado respetando la cantidad de espacio máximo por cada ítem. Nombrarlo siguiendo el formato `GG_Nombre_de_grupo`
    - Adjuntar el notebook final en formatos .pdf e .ipynb. Es necesario que los resultados puedan reproducirse al ejecutar todas las celdas en orden (verificarlo haceindo: Kernel -> Restart and Run All). 
    - Las predicciones del *held out* del punto 5 en formato csv.
- Habŕa una entrega intermedia obligatoria que deberán hacer antes del 25 de abril de 2024 a las 23:55hs. Para esta entrega deberán enviar el código que resuelve los primeros 3 ejercicios. 
- La **fecha** y **hora límite** de entrega está determinada en el campus de la materia.
- El trabajo deberá elaborarse en grupos de hasta 4 personas (4 preferentemente).
- Se podrán pedir pruebas de integridad y autoría; es decir, verificar que la salida solicitada es fruto del modelo presentado y que el modelo fue construido según lo requerido en este enunciado.
- La evaluación será grupal y se basará en la calidad del informe (presentación, claridad, prolijidad); la originalidad, practicidad y coherencia técnica de la solución; la corrección y solidez de las pruebas realizadas.


### Importante: sobre el uso de ChatGPT y grandes modelos de lenguaje

En este trabajo no estará explícitamente prohibido pero si fuertemente desaconsejado, consideramos a este trabajo práctico una importante herramienta de aprendizaje donde el uso de GPT puede ser perjudicial. En caso de usarlo se pide aclararlo en el informe y especificar cómo y en donde se utilizó. Así como expresar su opinión sobre la respuesta generada por el modelo pudiendo estar a favor o en contra de lo propuesto por este. Pueden adjuntar el link a la conversación con el modelo.

**Nota**: Agradecemos a [Martín García Sola](https://ar.linkedin.com/in/martin-e-garcia-sola) por la asistencia biológica en la confección de este Trabajo Práctico.